In [ ]:
import pandas as pd
from supabase import create_client
from pymongo import MongoClient

pd.set_option('display.max_columns', None)

In [ ]:
# conexión a la base de datos operacional
url_operacional = 'https://ggvtnhsokxrroymxgres.supabase.co'
key_operacional = ''
supabase_op = create_client(url_operacional, key_operacional)

# conexión a la base de datos de MongoDB
uri = ''
client = MongoClient(uri)
mongo = client['chauflix']

# conexión al datawarehouse
url_dw = 'https://hcwyzlprqjlwqwdrfrco.supabase.co'
key_dw = ''
supabase_dw = create_client(url_dw, key_dw)

In [111]:
users = supabase_op.schema("public").table("user").select("*").execute()

pd.DataFrame(users.data)

,id,email,first_name,last_name,birth_date,gender,country_id
0,1,bensonjacob@yahoo.com,Jared,Edwards,1988-07-25,Male,17
1,2,mendozawilliam@jones-perry.org,Kara,Nelson,1980-07-14,Male,16
2,3,aaron74@nelson.com,Frank,Hines,1967-08-09,Male,39
3,4,scottjohn@hotmail.com,Kristina,Gonzalez,1973-07-30,Female,9
4,5,mitchelldonald@williams.com,Jaclyn,Fields,1985-11-14,Female,17
...,...,...,...,...,...,...,...
605,606,melissahicks@hotmail.com,Lindsay,Smith,1974-03-10,Female,8
606,607,moralestony@yahoo.com,Allen,Sandoval,2002-02-02,Other,4
607,608,pmccormick@holt.biz,Stephen,Morton,1964-09-25,Other,28
608,609,hayesderek@hotmail.com,Scott,Martinez,1968-07-08,Male,4


### dim_movies

In [112]:
movies = mongo["movies"]

movies_df = pd.DataFrame(list(movies.find()))
movies_df.head()

,_id,id,title,genres,release_year,duration_min
0,6840dc404822cb9ccf214518,5,Father of the Bride Part II,[Comedy],1995,153
1,6840dc404822cb9ccf21451c,9,Sudden Death,[Action],1995,109
2,6840dc404822cb9ccf214521,14,Nixon,[Drama],1995,180
3,6840dc404822cb9ccf214524,17,Sense and Sensibility,"[Drama, Romance]",1995,138
4,6840dc404822cb9ccf21452e,27,Now and Then,"[Children, Drama]",1995,83


In [113]:
movies_df.drop(columns=['_id', 'release_year', 'duration_min'], inplace=True)

movies_df.head()

,id,title,genres
0,5,Father of the Bride Part II,[Comedy]
1,9,Sudden Death,[Action]
2,14,Nixon,[Drama]
3,17,Sense and Sensibility,"[Drama, Romance]"
4,27,Now and Then,"[Children, Drama]"


In [114]:
licenses = supabase_op.schema("public").table("license").select("*").execute()
licenses_df = pd.DataFrame(licenses.data)
licenses_df.head()

,id,movie_id,studio_id,price,duration,type_id
0,1,1,None,108809.0,300,1
1,2,2,None,45218.0,240,2
2,3,3,None,106708.0,365,1
3,4,4,None,92798.0,300,1
4,5,5,None,44595.0,120,2


In [115]:
licenses_payment = supabase_op.schema("public").table("license_payment").select("*").execute()
licenses_payment_df = pd.DataFrame(licenses_payment.data)

merge = licenses_df.merge(licenses_payment_df, left_on='id', right_on='license_id', how='left')

In [116]:
merge = merge[['movie_id', 'duration', 'date']]

In [117]:
merge.head()

,movie_id,duration,date
0,1,300,2024-11-02
1,2,240,2025-06-02
2,3,365,2024-06-09
3,4,300,2025-02-17
4,5,120,2025-04-13


In [118]:
for row in merge.itertuples():
    movie_id = row.movie_id
    duration = row.duration
    date = row.date

    date = pd.to_datetime(date).date()
    expiration_date = date + pd.Timedelta(days=duration)
    expiration_date = expiration_date.strftime('%Y-%m-%d')

    movies_df.loc[movies_df['id'] == movie_id, 'license_expiration'] = expiration_date

In [119]:
movies_df.head()

,id,title,genres,license_expiration
0,5,Father of the Bride Part II,[Comedy],2025-08-11
1,9,Sudden Death,[Action],2025-07-05
2,14,Nixon,[Drama],2025-12-17
3,17,Sense and Sensibility,"[Drama, Romance]",2025-06-10
4,27,Now and Then,"[Children, Drama]",2025-06-10


In [120]:
rating = mongo["ratings"]
rating_df = pd.DataFrame(list(rating.find()))

In [121]:
rating_df.drop(columns=['_id'], inplace=True)
rating_df.head()

,user_id,movie_id,rating,timestamp
0,1,6,4.0,2000-07-30T18:37:04Z
1,1,326,4.0,2000-07-30T18:28:30Z
2,1,399,5.0,2000-07-30T18:31:49Z
3,1,560,3.0,2000-07-30T18:29:35Z
4,1,595,3.0,2000-07-30T18:44:13Z


In [122]:
rating_df['rating'] = rating_df['rating']/5*10
rating_df.head()

,user_id,movie_id,rating,timestamp
0,1,6,8.0,2000-07-30T18:37:04Z
1,1,326,8.0,2000-07-30T18:28:30Z
2,1,399,10.0,2000-07-30T18:31:49Z
3,1,560,6.0,2000-07-30T18:29:35Z
4,1,595,6.0,2000-07-30T18:44:13Z


In [123]:
for row in movies_df.itertuples():
    movie_id = row.id
    
    mean_rating = rating_df[rating_df['movie_id'] == movie_id]['rating'].mean()

    movies_df.loc[movies_df['id'] == movie_id, 'rating'] = mean_rating

In [124]:
movies_df.head()

,id,title,genres,license_expiration,rating
0,5,Father of the Bride Part II,[Comedy],2025-08-11,6.142857
1,9,Sudden Death,[Action],2025-07-05,6.250000
2,14,Nixon,[Drama],2025-12-17,7.666667
3,17,Sense and Sensibility,"[Drama, Romance]",2025-06-10,7.552239
4,27,Now and Then,"[Children, Drama]",2025-06-10,6.666667


In [127]:
for row in movies_df.itertuples():
    genres = row.genres
    
    if genres:
        primer_genero = genres[0]
    else:
        primer_genero = None

    movies_df.loc[movies_df['id'] == row.id, 'genres'] = primer_genero

In [129]:
movies_df.head()

,id,title,genres,license_expiration,rating
0,5,Father of the Bride Part II,Comedy,2025-08-11,6.142857
1,9,Sudden Death,Action,2025-07-05,6.250000
2,14,Nixon,Drama,2025-12-17,7.666667
3,17,Sense and Sensibility,Drama,2025-06-10,7.552239
4,27,Now and Then,Children,2025-06-10,6.666667


In [ ]:
movies_df.rename(columns={'genres': 'genre', 'rating': 'mean_rating'}, inplace=True)

In [ ]:
movies_df.dropna(inplace=True) # polemico, no me dejaba subir un na

In [135]:
insert = supabase_dw.schema("public").table("dim_movie").insert(movies_df.to_dict(orient='records')).execute()